<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularization

*Author: Alexander Del Toro Barba*

## Overview

* Regularization is a technique for preventing over-fitting by penalizing a model for having large weights
* Weight regularization was borrowed from penalized regression models in statistics

## L1 (Lasso)

* Sum of the absolute weights


## L2 (Ridge)

* Sum of the squared weights
* The most common type of regularization is L2, also called simply “weight decay,” with values often on a logarithmic scale between 0 and 0.1, such as 0.1, 0.001, 0.0001, etc.

## Elastic Net

* Linear combination of L1 and L2

## Lambda Value

* Lambda is a regularization hyperparameter
* Reasonable values of lambda range between 0 and 0.1
* L2 weight regularization with very small regularization hyperparameters such as (e.g. 0.0005 or 5 x 10^−4) may be a good starting point

# RNN Model

## Import & Prepare Data

In [0]:
import tensorflow as tf
import datetime, os

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Choose Regularization

The Dense layer takes three regularizers, which all default to None. 
* kernel_regularizer: Regularizer function applied to the kernel weights matrix.
* bias_regularizer: Regularizer function applied to the bias vector.
* activity_regularizer: Regularizer function applied to the output of the layer (its "activation")

In [0]:
kernel_regularizer=tf.keras.regularizers.l2(l=0.0005)
bias_regularizer=None
activity_regularizer=None

## Define Model & Run

In [7]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(512, activation='relu', 
                                kernel_regularizer=kernel_regularizer, 
                                bias_regularizer=bias_regularizer, 
                                activity_regularizer=activity_regularizer))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=5, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.6831 - acc: 0.8137 - val_loss: 0.6109 - val_acc: 0.8162
Epoch 2/5
60000/60000 [==============================] - 5s 76us/sample - loss: 0.5375 - acc: 0.8428 - val_loss: 0.5487 - val_acc: 0.8322
Epoch 3/5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.5099 - acc: 0.8490 - val_loss: 0.5410 - val_acc: 0.8361
Epoch 4/5
60000/60000 [==============================] - 5s 81us/sample - loss: 0.4953 - acc: 0.8524 - val_loss: 0.4899 - val_acc: 0.8572
Epoch 5/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.4853 - acc: 0.8546 - val_loss: 0.5258 - val_acc: 0.8373
